In [1]:
import pandas as pd
import numpy as np

import seaborn as sns

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

C:\Users\Abhishek\Anaconda2\lib\site-packages\matplotlib\__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
train = pd.read_csv('./data/train.csv', index_col='id', encoding="ISO-8859-1")
test = pd.read_csv('./data/test.csv', index_col='id', encoding="ISO-8859-1")

products_description = pd.read_csv('./data/product_descriptions.csv')
products_attributes = pd.read_csv('./data/attributes.csv')

In [3]:
# merge datasets
train_merged = pd.merge(train, products_description, how='left', on='product_uid')
test_merged = pd.merge(test, products_description, how='left', on='product_uid')

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

In [33]:
vec = CountVectorizer()
bow = vec.fit_transform(train_merged.search_term)

In [37]:
import nltk

In [52]:
search_terms_text = ' '.join(train_merged.search_term)
search_terms = search_term_text.split()

In [53]:
search_terms_freq = nltk.FreqDist(search_terms)

In [54]:
search_terms_freq.most_common(200)

[(u'door', 2643),
 (u'light', 1976),
 (u'for', 1803),
 (u'in', 1657),
 (u'inch', 1485),
 (u'wall', 1423),
 (u'shower', 1421),
 (u'with', 1416),
 (u'white', 1351),
 (u'wood', 1317),
 (u'x', 1302),
 (u'tile', 1288),
 (u'and', 1156),
 (u'paint', 1112),
 (u'cabinet', 1061),
 (u'outdoor', 1033),
 (u'water', 1015),
 (u'vanity', 956),
 (u'kitchen', 949),
 (u'bathroom', 946),
 (u'doors', 941),
 (u'gas', 916),
 (u'lights', 840),
 (u'air', 821),
 (u'led', 802),
 (u'sink', 786),
 (u'2', 780),
 (u'1/2', 772),
 (u'ceiling', 772),
 (u'faucet', 766),
 (u'kit', 760),
 (u'4', 733),
 (u'floor', 728),
 (u'saw', 714),
 (u'pipe', 712),
 (u'black', 709),
 (u'steel', 699),
 (u'fan', 696),
 (u'window', 684),
 (u'lighting', 668),
 (u'electric', 664),
 (u'1', 637),
 (u'glass', 622),
 (u'metal', 603),
 (u'heater', 596),
 (u'3', 588),
 (u'toilet', 580),
 (u'wire', 568),
 (u'6', 564),
 (u'pvc', 564),
 (u'12', 560),
 (u'patio', 548),
 (u'fence', 538),
 (u'cover', 531),
 (u'drill', 527),
 (u'plastic', 521),
 (u'tub'

In [45]:
titles_freq = nltk.FreqDist(train_merged.product_title)

In [51]:
titles_freq['Lithonia Lighting All Season 4 ft. 2-Light Grey T8 Strip Fluorescent Shop Light']

21

In [46]:
titles_freq.most_common(200)

[(u'Pressure-Treated Timber #2 Southern Yellow Pine (Common: 4 in. x 4 in. x 8 ft.; Actual: 3.56 in. x 3.56 in. x 96 in.)',
  21),
 (u'Lithonia Lighting All Season 4 ft. 2-Light Grey T8 Strip Fluorescent Shop Light',
  21),
 (u'2 in. x 4 in. x 96 in. Premium Kiln-Dried Whitewood Stud', 18),
 (u'Ryobi ONE+ 18-Volt Lithium-Ion Ultimate Combo Kit (6-Tool)', 17),
 (u'Ryobi ONE+ 18-Volt Lithium-Ion Cordless Drill/Driver and Impact Driver Kit (2-Tool)',
  17),
 (u'Custom Building Products VersaBond Gray 50 lb. Fortified Thin-Set Mortar',
  17),
 (u'Whirlpool Gold Series Top Control Dishwasher in Monochromatic Stainless Steel with Silverware Spray',
  15),
 (u'WeatherShield 2 in. x 4 in. x 8 ft. #2 Prime Prime Pressure-Treated Lumber',
  14),
 (u'23/32 in. x 4 ft. x 8 ft. RTD Sheathing Syp', 14),
 (u'Oriented Strand Board (Common: 7/16 in. x 4 ft. x 8 ft.; Actual: 0.418 in. x 47.75 in. x 95.75 in.)',
  14),
 (u'Ryobi ONE+ 18-Volt Lithium-Ion Cordless Super Combo Kit (4-Piece)', 14),
 (u'Owens

In [50]:
title = 'Whirlpool Gold Series Top Control Dishwasher in Monochromatic Stainless Steel with Silverware Spray'
train_merged[train_merged.product_title == title].relevance

6536    1.33
6537    2.33
6538    1.67
6539    2.33
6540    2.00
6541    2.33
6542    2.33
6543    2.67
6544    1.67
6545    2.00
6546    2.00
6547    1.33
6548    1.67
6549    1.67
6550    2.33
Name: relevance, dtype: float64